# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [50]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [51]:
import json
import numpy as np

## Settings / Constants

In [52]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama3_70B", "GPT-3"]
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment",
              "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ],
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ]}

In [53]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}",
                "Llama2_70B": "\\textbf{Llama-2-70B}", "Llama3_70B": "\\textbf{Llama-3-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

N_METRICS = {"aspect_category": 6, "aspect_category_sentiment": 6,
             "end_2_end_absa": 6, "target_aspect_sentiment_detection": 5}

## Code

In [54]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [55]:
def get_metric(metric_name, results):
    main_metric = add_thousand_dots(round_number(results[metric_name]*100, 2))
    std_metric = round_number(np.std([res[metric_name] * 100 for res in results["single_split_results"]]), 2)
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [56]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n\n", "-------#-----#-----#-------")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)} & {get_metric('eval_accuracy', results)} \\\\")
        else:
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1', results)} & {get_metric('eval_f1_macro', results)} & {get_metric('eval_accuracy', results)} \\\\")
    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(["fixed", "random"]):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)}"
                else:
                    f1_metrics = f"{get_metric('eval_f1', results)} & {get_metric('eval_f1_macro', results)}"
                print(
                    f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} & {get_metric('eval_accuracy', results)} \\\\")

                # print(llm_idx, fs_idx, freq_idx)
            if freq_idx == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{2-"+str(7)+"}\\arrayrulecolor{black}")
            else:
                print("\\hline")
        print("\hline")
    print("\n\n")

aspect_category :

 -------#-----#-----#-------
\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 90.90\textsubscript{1.37} & 89.97\textsubscript{1.66} & 83.80\textsubscript{2.01} \\
 &  & 1,000 & 0 & 92.02\textsubscript{1.19} & 91.10\textsubscript{1.46} & 86.07\textsubscript{1.63} \\
 &  & 2,000 & 0 & 92.35\textsubscript{1.15} & 91.53\textsubscript{1.31} & 86.23\textsubscript{1.86} \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 81.33\textsubscript{0.36} & 80.37\textsubscript{1.30} & 68.37\textsubscript{1.69} \\
 &  & 25 & 975 & 80.76\textsubscript{2.03} & 80.18\textsubscript{1.65} & 68.20\textsubscript{1.52} \\
 &  & 25 & 1,975 & 80.65\textsubscript{1.77} & 80.20\textsubscript{1.11} & 68.10\textsubscript{1.53} \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 89.88\textsubscript{1.42} & 88.77\textsubscript{1.72} & 82.27\textsu

### Report Metrics Fine-Grained Metrics

#### ACD

In [57]:
absa_task = "aspect_category"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
        for metric in ["f1", "accuracy"]:
            class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)

            if fs_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                    class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if fs_idx == 0 and freq_idx == 2:
               print(
                "\\arrayrulecolor{gray}\cline{2-14}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 85.23\textsubscript{3.71} &93.00\textsubscript{1.60} &93.57\textsubscript{0.90} &94.47\textsubscript{0.78} &93.03\textsubscript{1.84} &95.43\textsubscript{1.35} &87.25\textsubscript{3.80} &96.43\textsubscript{1.16} &90.78\textsubscript{3.57} &98.53\textsubscript{0.60}  \\
 &  & 1,000 & 0 & 86.68\textsubscript{2.65} &93.53\textsubscript{1.30} &94.71\textsubscript{0.92} &95.47\textsubscript{0.85} &94.15\textsubscript{1.33} &96.17\textsubscript{0.93} &88.21\textsubscript{2.87} &96.70\textsubscript{0.89} &91.74\textsubscript{2.42} &98.70\textsubscript{0.47}  \\
 &  & 2,000 & 0 & 87.41\textsubscript{2.77} &93.80\textsubscript{1.18} &94.92\textsubscript{0.82} &95.67\textsubscript{0.76} &94.41\textsubscript{1.10} &96.37\textsubscript{0.79} &88.34\textsubscript{2.96} &96.70\textsubscript{0.98} &92.56\textsubscript{3.00} &98.83\textsubscript{0.53}  \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{L

#### End-2-End ABSA

In [58]:
absa_task = "end_2_end_absa"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for polarity in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
        for metric in ["f1", "accuracy"]:
            class_wise_metrics += f"{get_metric(f'eval_{metric}_{polarity}', results)} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)

            if fs_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for polarity in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
                for metric in ["f1", "accuracy"]:
                    class_wise_metrics += f"{get_metric(f'eval_{metric}_{polarity}', results)} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if fs_idx == 0 and freq_idx == 2:
               print(
                "\\arrayrulecolor{gray}\cline{2-10}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 82.11\textsubscript{2.60} &69.74\textsubscript{3.80} &71.67\textsubscript{2.53} &55.90\textsubscript{3.08} &56.43\textsubscript{16.42} &40.98\textsubscript{14.69}  \\
 &  & 1,000 & 0 & 84.44\textsubscript{2.91} &73.18\textsubscript{4.43} &76.71\textsubscript{1.73} &62.25\textsubscript{2.28} &63.94\textsubscript{7.56} &47.43\textsubscript{7.84}  \\
 &  & 2,000 & 0 & 85.03\textsubscript{3.83} &74.15\textsubscript{5.78} &78.18\textsubscript{4.12} &64.35\textsubscript{5.37} &73.29\textsubscript{9.80} &58.84\textsubscript{12.83}  \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 60.42\textsubscript{3.22} &43.36\textsubscript{3.32} &56.69\textsubscript{3.00} &39.62\textsubscript{2.90} &16.40\textsubscript{5.84} &9.05\textsubscript{3.61}  \\
 &  & 25 & 975 & 60.53\textsubscript{4.78} &43.56\textsubscript{4.92} &60.01\textsubscript{1.82} &42.89\textsubscript{1.8

#### ACSA

In [59]:
def filter_str(values):
    return [value for value in values if isinstance(value, (int, float))]

def get_metric(metric, ac, polarity, results):
    metric_name = f'{metric}_{ac}-{polarity}'
    n_examples_field = f'eval_n_examples_{ac}-{polarity}'
    main_metric = add_thousand_dots(round_number(results[metric_name]*100, 2))
    std_metric = round_number(np.std(
        filter_str([res[metric_name] * 100 for res in results["single_split_results"]])), 2)
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [60]:
absa_task = "aspect_category_sentiment"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 10 if ac_idx == 2 else 16
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 82.88\textsubscript{3.05} & 95.77\textsubscript{0.41} & 0.00\textsubscript{0.00} & 99.17\textsubscript{0.33} & 82.18\textsubscript{7.34} & 96.33\textsubscript{1.48} & 91.93\textsubscript{1.48} & 96.13\textsubscript{0.84} & 66.30\textsubscript{8.56} & 98.07\textsubscript{0.49} & 82.17\textsubscript{4.06} & 94.37\textsubscript{1.19} \\
 &  & 1,000 & 0 & 84.74\textsubscript{2.43} & 96.07\textsubscript{0.56} & 57.12\textsubscript{26.45} & 99.43\textsubscript{0.45} & 85.18\textsubscript{6.61} & 96.93\textsubscript{1.30} & 94.47\textsubscript{1.10} & 97.33\textsubscript{0.57} & 81.56\textsubscript{5.95} & 98.63\textsubscript{0.41} & 86.87\textsubscript{2.86} & 95.70\textsubscript{0.93} \\
 &  & 2,000 & 0 & 85.57\textsubscript{3.62} & 96.27\textsubscript{0.81} & 67.67\textsubscript{20.57} & 99.50\textsubscript{0.40} & 86.50\textsubscript{4.17} & 97.07\textsubscript{0.96} & 94.

#### TASD

In [61]:
def filter_str(values):
    return [value for value in values if isinstance(value, (int, float))]

def get_metric(metric, ac, polarity, results):
    metric_name = f'{metric}_{ac}_{polarity}'
    n_examples_field = f'eval_n_examples_{ac}-{polarity}'
    main_metric = add_thousand_dots(round_number(results[metric_name]*100, 2))
    std_metric = round_number(np.std(
        filter_str([res[metric_name] * 100 for res in results["single_split_results"]])), 2)
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [62]:
absa_task = "target_aspect_sentiment_detection"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 10 if ac_idx == 2 else 16
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 70.00\textsubscript{7.04} & 53.85\textsubscript{7.63} & 28.57\textsubscript{26.07} & 16.67\textsubscript{22.07} & 51.85\textsubscript{7.86} & 35.00\textsubscript{8.52} & 67.91\textsubscript{4.46} & 51.42\textsubscript{5.24} & 72.22\textsubscript{11.66} & 56.52\textsubscript{12.06} & 45.90\textsubscript{3.61} & 29.79\textsubscript{3.24} \\
 &  & 1,000 & 0 & 69.63\textsubscript{5.18} & 53.41\textsubscript{6.51} & 50.00\textsubscript{15.75} & 33.33\textsubscript{15.82} & 50.00\textsubscript{8.50} & 33.33\textsubscript{9.12} & 75.93\textsubscript{3.25} & 61.19\textsubscript{3.93} & 87.80\textsubscript{11.98} & 78.26\textsubscript{14.24} & 47.92\textsubscript{2.32} & 31.51\textsubscript{2.10} \\
 &  & 2,000 & 0 & 71.11\textsubscript{4.02} & 55.17\textsubscript{5.16} & 57.14\textsubscript{23.40} & 40.00\textsubscript{17.39} & 54.90\textsubscript{8.03} & 37.84\textsubscript{9.